In [1]:
import re, math
from collections import Counter
import string
import nltk
from difflib import SequenceMatcher
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk import word_tokenize 
#nltk.download('punkt')


[nltk_data] Downloading package stopwords to C:\Users\Vashi
[nltk_data]     NSIT\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [1]:
import operator
import xgboost as xgb
from xgboost.sklearn import XGBClassifier

In [2]:
import pandas as pd
import numpy as np
import random as rnd

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing


from sklearn.cross_validation import train_test_split , StratifiedKFold
from sklearn.feature_selection import RFECV

import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab


F:\Anaconda\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [5]:
def describe_more( df ):
    var = [] ; l = [] ; t = []
    for x in df:
        var.append( x )
        l.append( len( pd.value_counts( df[ x ] ) ) )
        t.append( df[ x ].dtypes )
    #print(var,l,t)
    levels = pd.DataFrame( { 'Variable' : var , 'Levels' : l , 'Datatype' : t } )
    levels.sort_values( by = 'Levels' , inplace = True )
    return levels

In [272]:
label=preprocessing.LabelEncoder()

In [ ]:
camp=pd.read_csv("campaign_data.csv")

In [150]:
camp1=pd.read_csv("camp.csv")

In [8]:
WORD = re.compile(r'\w+')

def get_cosine(vec1, vec2):
     intersection = set(vec1.keys()) & set(vec2.keys())
        
     numerator = sum([vec1[x] * vec2[x] for x in intersection])

     sum1 = sum([vec1[x]**2 for x in vec1.keys()])
     sum2 = sum([vec2[x]**2 for x in vec2.keys()])
     denominator = math.sqrt(sum1) * math.sqrt(sum2)

     if not denominator:
        return 0.0
     else:
        return float(numerator) / denominator

def text_to_vector(text):
     words = WORD.findall(text)
     return Counter(words)
def words(scent):
    word=scent.lower()
    word=word_tokenize(word)
    #word= re.findall(r'\w+',scent)
    exclude= list(filter(lambda x: x in ['what','where','how','when','why','?'],word))
    word=list(filter(lambda x: x not in (stopwords.words('english') + list(string.punctuation)), word))
    word.extend(exclude)
    return word


In [9]:
def cosine_similarity(scent,scent1):
    vector1 = text_to_vector(scent)
    vector2 = text_to_vector(scent1)
    cosine = get_cosine(vector1, vector2)
    return cosine

def sequential(scent,scent1):
    s1=words(scent)
    s2=words(scent1)
    s = SequenceMatcher(None, s1, s2)
    print(s.ratio())
    return s.value
def intersection(scent,scent1):
    scent=words(scent)
    scent1=words(scent1)
    s1=set(scent)
    s2=set(scent1)
    s=s1.intersection(s2)
    return len(s)

In [10]:
data=pd.read_csv('train.csv')

In [11]:
data.head()

,id,user_id,campaign_id,send_date,is_open,is_click
0,42_14051,14051,42,01-09-2017 19:55,0,0
1,52_134438,134438,52,02-11-2017 12:53,0,0
2,33_181789,181789,33,24-07-2017 15:15,0,0
3,44_231448,231448,44,05-09-2017 11:36,0,0
4,29_185580,185580,29,01-07-2017 18:01,0,0


In [3]:
camp=pd.read_csv('campfinal.csv')
camp.head()  #campaign data after adding weihgts in most clicked order #1st iteration for calculating weights

,Unnamed: 0,campaign_id,communication_type,total_links,no_of_internal_links,no_of_images,no_of_sections,email_body,subject,email_url,campaign_wts,campaign_wts1,images,internal_links,sections,links
0,0,29,Newsletter,67,61,12,3,"Dear AVians,\r\r\r\r\n \r\r\r\r\nWe are shapin...",Sneak Peek: A look at the emerging data scienc...,http://r.newsletters.analyticsvidhya.com/7um44...,6,6,"(10.0, 19.0]","(26.0, 191.0]","(2.0, 6.0]","(28.0, 203.0]"
1,1,30,Upcoming Events,18,14,7,1,"Dear AVians,\r\r\r\r\n \r\r\r\r\nAre your eage...",[July] Data Science Expert Meetups & Competiti...,http://r.newsletters.analyticsvidhya.com/7up0e...,5,5,"(3.0, 10.0]","(7.0, 26.0]","(0.999, 2.0]","(9.0, 28.0]"
2,2,31,Conference,15,13,5,1,Early Bird Pricing Till August 07  Save upto ...,Last chance to convince your boss before the E...,http://r.newsletters.analyticsvidhya.com/7usym...,2,2,"(3.0, 10.0]","(7.0, 26.0]","(0.999, 2.0]","(9.0, 28.0]"
3,3,32,Conference,24,19,7,1,\r\r\r\r\n \r\r\r\r\nHi ?\r\r\r\r\n \r\r\r\r\n...,A.I. & Machine Learning: 5 reasons why you sho...,http://r.newsletters.analyticsvidhya.com/7uthl...,5,5,"(3.0, 10.0]","(7.0, 26.0]","(0.999, 2.0]","(9.0, 28.0]"
4,4,33,Others,7,3,1,1,Fireside Chat with DJ Patil - the master is he...,"[Delhi NCR] Fireside Chat with DJ Patil, Forme...",http://r.newsletters.analyticsvidhya.com/7uvlg...,4,4,"(0.999, 3.0]","(1.999, 7.0]","(0.999, 2.0]","(5.999, 9.0]"


In [593]:
def order(data,output,on,by):
    D=data[data[output]==on].groupby([output,by])[by].agg({'Frequency':'count'}).sort_values(by='Frequency',ascending=False)
    return D

In [13]:
C_cid=order(data,'is_click',1,'campaign_id')

F:\Anaconda\lib\site-packages\ipykernel\__main__.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  from ipykernel import kernelapp as app


In [323]:
U_C_data=data[data['is_click']==1]

In [336]:
category={}
for i in camp.communication_type.unique():
    typ=list(camp.campaign_id[camp['communication_type']==i].values)
    category[i]=typ
    

In [342]:
category

{'Conference': [31, 32, 36, 37, 38, 40, 49, 53],
 'Corporate': [41, 43, 55, 59, 60, 61, 66, 70, 72, 73, 77, 80],
 'Hackathon': [45, 46, 47, 51, 58, 62, 65, 67, 68, 69, 71, 78, 79],
 'Newsletter': [29, 34, 42, 48, 52, 54, 56, 63, 74],
 'Others': [33, 35],
 'Upcoming Events': [30, 44, 50, 57, 64, 75, 76],
 'Webinar': [39]}

In [ ]:
U_C_data['category']=0

In [343]:
for key, val in category.items():
    for i in val:
        U_C_data.category[U_C_data['campaign_id']==i]=key

F:\Anaconda\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
F:\Anaconda\lib\site-packages\pandas\core\generic.py:5984: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
F:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2881: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
F:\Anaconda\lib\site-packages\ipykernel\__main__.py:3: Setting

In [362]:
U_C_corr=order1(U_C_data,'category','user_id')

In [717]:
Dat=data

In [380]:
uc_corr=U_C_corr[['category','user_id']]

In [973]:
for i in uc_corr['category'].unique():
    uc=uc_corr[uc_corr['category']==i]
    data=data.join(uc,on='user_id')
    data[i]=data['category']
    data.drop('category', axis=1, inplace=True)

In [974]:
for i in uc_corr['category'].unique():
    uc=uc_corr[uc_corr['category']==i]
    test=test.join(uc,on='user_id')
    test[i]=test['category']
    test.drop('category', axis=1, inplace=True)

In [975]:
data.to_csv('Trainfinal.csv')

In [976]:
test.to_csv('Testfinal.csv')

In [1060]:
test=pd.read_csv('Testfinal.csv')

In [ ]:
Dat.loc[Dat['Corporate']=='Corporate','Corporate']

In [984]:
category

{'Conference': [31, 32, 36, 37, 38, 40, 49, 53],
 'Corporate': [41, 43, 55, 59, 60, 61, 66, 70, 72, 73, 77, 80],
 'Hackathon': [45, 46, 47, 51, 58, 62, 65, 67, 68, 69, 71, 78, 79],
 'Newsletter': [29, 34, 42, 48, 52, 54, 56, 63, 74],
 'Others': [33, 35],
 'Upcoming Events': [30, 44, 50, 57, 64, 75, 76],
 'Webinar': [39]}

In [1206]:
for i,j in category.items():
    data.loc[data[i].isnull()& data['U_cid_wts']>0,i]=0    

In [1210]:
for i,j in category.items():
    data.loc[data[i]==i,i]=1    

In [1216]:
data.to_csv('Data_FinaL.csv')

In [1212]:
Data=data.set_index('user_id',drop=False)

In [ ]:
for j,k in category.items():
    for i in UniqueIds_click:
        if i in .index:
            try:    
                if Data[j][i].any()==j:
                    Data[j][i]=1
                else:
                    Data[j][i]=0
            except:
                if Data[j][i]==j:
                    Data[j][i]=1
                else:
                    Data[j][i]=0

In [1251]:
data.to_csv('Data_Final.csv')

In [1014]:
ClickIds=data.loc[data['is_click']==1,'user_id'].values

In [1020]:
len(ClickIds)

12782

In [1021]:
UniqueIds_click=np.unique(ClickIds)

In [1030]:
Test.drop(['Conference','Corporate','Hackathon','Newsletter','Others','Upcoming Events','Webinar'],axis=1,inplace=True)

In [1066]:
Test=Test.set_index('user_id',drop=False)

In [1040]:
Data[Data.user_id.unique()]

array([ 62110,  73007,   8759, ..., 103245,  36187, 235406], dtype=int64)

In [1052]:
B=pd.DataFrame({'bc':[1,2,3,4,5,6],'C':['a','b','c','d','d','d'],'D':[0,'b',0,'c','c','c']})

In [1053]:
B=B.set_index('bc',drop=True)

In [ ]:
['Conference','Corporate','Hackathon','Newsletter','Others','Upcoming Events','Webinar']

In [ ]:
Data=data[['Conference','Corporate','Hackathon','Newsletter','Others','Upcoming Events','Webinar']]

In [1129]:
Test.to_csv('Test_finaL.csv')

In [1128]:
for j,k in category.items():
    for i in UniqueIds_click:
        Test.loc[Test[j].isnull()&Test['user_id']==i,j]=0

In [1195]:
for j,k in category.items():
    for i in UniqueIds_click:
        if i in Test.index:
            try:    
                if Test[j][i].any()==j:
                    Test[j][i]=1
                else:
                    Test[j][i]=0
            except:
                if Test[j][i]==j:
                    Test[j][i]=1
                else:
                    Test[j][i]=0


F:\Anaconda\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
F:\Anaconda\lib\site-packages\ipykernel\__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
F:\Anaconda\lib\site-packages\ipykernel\__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
F:\Anaconda\lib\site-packages\ipykernel\__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http:/

In [1169]:
Test['communication_type'][393]

'Newsletter'

In [1159]:
if Test.loc[95437,'Newsletter'].any()=='Newsletter':
    print('d')
else: print('e')

d


In [1196]:
Test.to_csv('TEST_final.csv')

In [1194]:
Test=Test.set_index('user_id',drop=False)

In [1221]:
test=test.set_index('Unnamed: 0',drop=True)

In [1250]:
test.to_csv('Test_FinaL.csv')

In [1243]:
test.communication_type.unique()

array(['Newsletter', 'Upcoming Events', 'Hackathon', 'Corporate'], dtype=object)

In [1245]:
test=pd.get_dummies(test,columns=['communication_type'])

In [1246]:
data=pd.get_dummies(data,columns=['communication_type'])

In [1248]:
test['communication_type_Others']=0
test['communication_type_Webinar']=0
test['communication_type_Conference']=0

In [1258]:
Test.to_csv('Test_FinaL_nonan.csv')

In [1256]:
Test=Test.fillna({'U_cid_wts':0,'U_oid_wts':0})

In [397]:
uc_corr=uc_corr.set_index('user_id')

In [398]:
Dat=Data.join(uc_corr,on='user_id')

In [367]:
ucdict={}
for i in U_C_corr['category'].unique():
    ucdict[i]=U_C_corr.user_id[U_C_corr['category']==i].values.tolist()

In [952]:
comm=pd.DataFrame(camp['communication_type'])

In [966]:
campwt=pd.DataFrame(camp['campaign_weight'])

In [962]:
data= data.join(comm, on='campaign_id')

In [968]:
data=data.join(campwt,on='campaign_id')

In [963]:
test=test.join(comm, on='campaign_id')

In [969]:
test=test.join(campwt,on='campaign_id')

In [355]:
U_C_corr.index.values
U_C_index=U_C_corr.index.tolist()

array([('Conference', 44), ('Conference', 112), ('Conference', 158), ...,
       ('Webinar', 235000), ('Webinar', 236832), ('Webinar', 238494)], dtype=object)

In [1242]:
camp # revised campaign weights under column=campaign_weight after steps given below, 

,campaign_id,communication_type,total_links,no_of_internal_links,no_of_images,no_of_sections,email_body,subject,email_url,campaign_wts,campaign_wts1,images,internal_links,sections,links,external_links,campaign_weight
campaign_id,,,,,,,,,,,,,,,,,
29,29,Newsletter,67,61,12,3,"Dear AVians,\r\r\r\n \r\r\r\nWe are shaping up...",Sneak Peek: A look at the emerging data scienc...,http://r.newsletters.analyticsvidhya.com/7um44...,6,6,"(10.0, 19.0]","(26.0, 191.0]","(2.0, 6.0]","(28.0, 203.0]",6,5.40
30,30,Upcoming Events,18,14,7,1,"Dear AVians,\r\r\r\n \r\r\r\nAre your eager to...",[July] Data Science Expert Meetups & Competiti...,http://r.newsletters.analyticsvidhya.com/7up0e...,5,5,"(3.0, 10.0]","(7.0, 26.0]","(0.999, 2.0]","(9.0, 28.0]",4,4.61
31,31,Conference,15,13,5,1,Early Bird Pricing Till August 07  Save upto ...,Last chance to convince your boss before the E...,http://r.newsletters.analyticsvidhya.com/7usym...,2,2,"(3.0, 10.0]","(7.0, 26.0]","(0.999, 2.0]","(9.0, 28.0]",2,1.91
32,32,Conference,24,19,7,1,\r\r\r\n \r\r\r\nHi ?\r\r\r\n \r\r\r\nBefore I...,A.I. & Machine Learning: 5 reasons why you sho...,http://r.newsletters.analyticsvidhya.com/7uthl...,5,5,"(3.0, 10.0]","(7.0, 26.0]","(0.999, 2.0]","(9.0, 28.0]",5,4.61
33,33,Others,7,3,1,1,Fireside Chat with DJ Patil - the master is he...,"[Delhi NCR] Fireside Chat with DJ Patil, Forme...",http://r.newsletters.analyticsvidhya.com/7uvlg...,4,4,"(0.999, 3.0]","(1.999, 7.0]","(0.999, 2.0]","(5.999, 9.0]",4,3.31
34,34,Newsletter,75,64,12,4,\r\r\r\nAugust Newsletter\r\r\r\n \r\r\r\nDear...,"[AV Newsletter] Fireside Chat with DJ Patil, D...",http://r.newsletters.analyticsvidhya.com/7uwwu...,5,5,"(10.0, 19.0]","(26.0, 191.0]","(2.0, 6.0]","(28.0, 203.0]",11,4.98
35,35,Others,9,2,1,1,Fireside Chat with DJ Patil - Watch Live!\r\r\...,Watch Live: Fireside Chat with DJ Patil,http://r.newsletters.analyticsvidhya.com/7ux6c...,2,2,"(0.999, 3.0]","(1.999, 7.0]","(0.999, 2.0]","(5.999, 9.0]",7,2.20
36,36,Conference,13,11,2,1,Announcing Dr. Kirk Borne as Keynote Speaker\r...,"Dr Kirk Borne of Booz Allen Hamilton, to keyno...",http://r.newsletters.analyticsvidhya.com/7uxpa...,3,3,"(0.999, 3.0]","(7.0, 26.0]","(0.999, 2.0]","(9.0, 28.0]",2,2.41
37,37,Conference,9,7,3,1,Last day to save maximum on tickets\r\r\r\n \r...,[Last Day] Early Bird Offer on conference pass...,http://r.newsletters.analyticsvidhya.com/7uyr8...,1,1,"(0.999, 3.0]","(1.999, 7.0]","(0.999, 2.0]","(5.999, 9.0]",2,1.41


In [353]:
camp.to_csv('campaignfinal.csv')

In [14]:
def freq_dist(feature,n):
    X=pd.DataFrame()
    X['Frequency_dist']=pd.qcut(feature['Frequency'],n)
    return pd.concat([feature,X],axis=1)


In [15]:
C_cid=freq_dist(C_cid,6)

In [8]:
camp_train=camp[0:26]

In [9]:
camp.corr()

,Unnamed: 0,campaign_id,total_links,no_of_internal_links,no_of_images,no_of_sections,campaign_wts,campaign_wts1
Unnamed: 0,1.000000,1.000000,-0.080930,-0.086421,0.041577,0.075829,0.160436,0.014246
campaign_id,1.000000,1.000000,-0.080930,-0.086421,0.041577,0.075829,0.160436,0.014246
total_links,-0.080930,-0.080930,1.000000,0.999027,0.694684,0.569531,0.507851,0.506749
no_of_internal_links,-0.086421,-0.086421,0.999027,1.000000,0.703417,0.572091,0.507902,0.500610
no_of_images,0.041577,0.041577,0.694684,0.703417,1.000000,0.756261,0.600772,0.395113
no_of_sections,0.075829,0.075829,0.569531,0.572091,0.756261,1.000000,0.552956,0.366322
campaign_wts,0.160436,0.160436,0.507851,0.507902,0.600772,0.552956,1.000000,0.688098
campaign_wts1,0.014246,0.014246,0.506749,0.500610,0.395113,0.366322,0.688098,1.000000


In [195]:
camp['images']=pd.qcut(camp['no_of_images'],3)

In [530]:
camp_train['images']=pd.qcut(camp_train['no_of_images'],4)

F:\Anaconda\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [538]:
camp_test['images']=pd.qcut(camp_test['no_of_images'],4)

F:\Anaconda\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [196]:
camp['internal_links']=pd.qcut(camp['no_of_internal_links'],3)

In [542]:
camp_test['internal_links']=pd.qcut(camp_test['no_of_internal_links'],4)

F:\Anaconda\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [513]:
camp_train['internal_links']=pd.qcut(camp_train['no_of_internal_links'],4)

F:\Anaconda\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [197]:
camp['sections']=pd.qcut(camp['no_of_sections'],3,duplicates='drop')

In [216]:
camp_train['sections']=pd.qcut(camp_train['no_of_sections'],3,duplicates='drop')

F:\Anaconda\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [540]:
camp_test['sections']=pd.qcut(camp_test['no_of_sections'],3,duplicates='drop')

F:\Anaconda\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [198]:
camp['links']=pd.qcut(camp['total_links'],3)

In [514]:
camp_train['links']=pd.qcut(camp_train['total_links'],4)

F:\Anaconda\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [541]:
camp_test['links']=pd.qcut(camp_test['total_links'],4)

F:\Anaconda\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [361]:
def order1(data,output,by):
    D=data.groupby([output,by],as_index=False)[by].agg({'Frequency':'count'})#.sort_values(by='Frequency',ascending=False)
    return D

In [519]:
def order1(data,output,bye):
    D=data[[output,bye]].groupby([output],as_index=False).mean().sort_values(by=bye,ascending=False)
    return D

In [ ]:
data.groupby(['time_of_day','campaign_wts'])['campaign_wts'].agg({'Frequency':'count'})

In [531]:
Cimg=order1(camp_train,'images','campaign_wts')

In [543]:
Cimgtest=order1(camp_test,'images','campaign_wts')

In [533]:
Cimg.reset_index(level=0,inplace=True)

In [580]:
Cimg

,index,images,campaign_wts
0,3,"(12.0, 19.0]",5.000000
1,2,"(6.0, 12.0]",4.750000
2,0,"(0.999, 2.0]",2.400000
3,1,"(2.0, 6.0]",1.333333


In [579]:
for i in Cimg['index']:
    Cimg.campaign_wts[Cimg['index']==i]

0    5.0
Name: campaign_wts, dtype: float64
1    4.75
Name: campaign_wts, dtype: float64
2    2.4
Name: campaign_wts, dtype: float64
3    1.333333
Name: campaign_wts, dtype: float64


In [630]:
camp['campaign_weight']=0

In [520]:
Clnk=order1(camp_train,'internal_links','campaign_wts')

In [655]:
Ctotlnk=order1(camp_train,'links','campaign_wts')

In [656]:
Ctotlnk.reset_index(level=0,inplace=True)

In [671]:
Ctotlnk

,index,links,campaign_wts
0,3,"(67.0, 203.0]",5.000000
1,2,"(16.5, 67.0]",4.714286
2,0,"(6.999, 9.0]",2.500000
3,1,"(9.0, 16.5]",1.600000


In [666]:
Clnk.reset_index(level=0,inplace=True)

In [668]:
Clnk

,index,internal_links,campaign_wts
0,3,"(61.75, 191.0]",5.142857
1,2,"(13.5, 61.75]",4.500000
2,0,"(1.999, 4.75]",2.714286
3,1,"(4.75, 13.5]",1.500000


In [277]:
Csec=order1(camp_train,'sections','campaign_wts')

In [669]:
Csec.reset_index(level=0,inplace=True)

In [670]:
Csec

,index,sections,campaign_wts
0,1,"(2.333, 6.0]",4.666667
1,0,"(0.999, 2.333]",2.882353


In [950]:
#names=['links','internal_links','images','sections'], Initial weights calculated below this code by using email texts/subs 
# and then iterated in this code to get the exact weights by using above information from images/links/sections etc

F=['total_links','no_of_internal_links','no_of_images','no_of_sections']
#def weights():
eps=0.01
for i in camp.campaign_id:
    val=camp.loc[camp['campaign_id']==i,F[0]].values    
    for n,k in enumerate(Ctotlnk['links']):
        try:
            if val> (k.left-eps)  and val< k.right :
                wt_1=Ctotlnk['campaign_wts'][n]
        except:
            if val==k.right:
                wt_1=Ctotlnk['campaign_wts'][n]
    val=camp.loc[camp['campaign_id']==i,F[1]].values
    for n,k in enumerate(Clnk['internal_links']):
        try:
            if val> (k.left-eps) and val< k.right:
                wt_2=Clnk['campaign_wts'][n]
        except:
            if val==k.right:
                wt_2=Clnk['campaign_wts'][n]
    val=camp.loc[camp['campaign_id']==i,F[2]].values
    for n,k in enumerate(Cimg['images']):
        try:
            if val> (k.left-eps) and val< k.right:
                wt_3=Cimg['campaign_wts'][n]
        except:
            if val==k.right:
                wt_3=Cimg['campaign_wts'][n]
    val=camp.loc[camp['campaign_id']==i,F[3]].values
    for n,k in enumerate(Csec['sections']):
        try:
            if val> k.left and val< k.right:
                wt_4=Csec['campaign_wts'][n]
        except:
            if val==k.right:
                wt_4=Csec['campaign_wts'][n]
    #print(wt_1,wt_2,wt_3,wt_4)
    camp.loc[camp['campaign_id']==i,'campaign_weight']=round((((wt_1+wt_2+wt_3+wt_4)/4)+(camp.campaign_wts[i]))/2,2)
    print('id'+str(i),camp.campaign_weight[i],camp.campaign_wts[i])


id29 5.4 6
id30 4.61 5
id31 1.91 2
id32 4.61 5
id33 3.31 4
id34 4.98 5
id35 2.2 2
id36 2.41 3
id37 1.41 1
id38 2.86 1
id39 2.07 2
id40 1.41 1
id41 1.41 1
id42 5.48 6
id43 1.41 1
id44 3.68 4
id45 2.68 3
id46 2.68 3
id47 2.68 3
id48 3.86 4
id49 5.25 6
id50 4.25 4
id51 2.18 2
id52 5.44 6
id53 4.75 5
id54 5.33 6
id55 2.91 4
id56 5.33 6
id57 4.36 4
id58 2.96 3
id59 3.18 4
id60 2.64 3
id61 3.11 3
id62 2.68 3
id63 5.48 6
id64 4.36 4
id65 2.81 3
id66 2.37 2
id67 2.81 3
id68 2.81 3
id69 2.41 3
id70 2.37 2
id71 2.68 3
id72 2.18 1
id73 1.8 1
id74 5.48 6
id75 5.36 6
id76 5.36 6
id77 3.68 4
id78 3.09 3
id79 3.09 3
id80 2.91 4


In [691]:
camp.to_csv('Campaign_Final.csv')  

In [21]:
def priority(feature,n=1): # set 0 for User ids esle 1 for Camp ids
    j=feature.columns.values[n]
    l=len(feature[j].unique())
    t={}
    for n,key in enumerate(feature[j].unique()):
        try:
            t[l-n]=[x[1] for x in (feature[feature[j]==key].index.tolist())]
        except:
            print('Error in adding to dict')
    return t

In [22]:
C_cid_feature=priority(C_cid)

In [280]:
C_cid_feature

{1: [40, 38, 37, 41, 43],
 2: [35, 51, 39, 31],
 3: [45, 36, 46, 47],
 4: [48, 50, 33, 44],
 5: [30, 53, 32, 34],
 6: [52, 29, 42, 49, 54]}

In [282]:
camp=camp.set_index('campaign_id',drop=False)

In [299]:
#without email body
text={}
trainer=[]
for key,val in C_cid_feature.items():
    for i in val:
        try:
            text[key].append(camp.subject[i]+' '+camp.communication_type[i])
        except:
            text[key]=[camp.subject[i],camp.communication_type[i]]
        trainer.append(i)

In [303]:
#with email body
text={}
trainer=[]
for key,val in C_cid_feature.items():
    for i in val:
        try:
            text[key].append(camp.subject[i]+' '+camp.communication_type[i]+' '+camp.email_body[i])
        except:
            text[key]=[camp.subject[i],camp.communication_type[i]+' '+camp.email_body[i]]
        trainer.append(i)

In [304]:
to_train=list(filter(lambda x: x not in trainer,camp.index.values))

In [308]:
#def weight_selector(trainer,to_trainer,camp):
wt={};wt1={}
for i in to_train:
    m={};n={}
    text1=camp.subject[i]+' '+camp.communication_type[i]+' '+camp.email_body[i]
    for key,val in text.items():
        l=[];c=[]
        for j in val:
            tot=intersection(text1,j)
            l.append(tot)
            cos=cosine_similarity(text1,j)
            c.append(cos)
        e=len(l)-1

        if l[e-1]==l[e] and (l[e]==1 or l[e]==2):
            if l[e-1]==l[e-2]:
                if l[e-2]==l[e-3]:
                    m[key]=e-3
                else: m[key]=e-2
            else: m[key]=e-1
        else:
            m[key]=max(l)
        n[key]=max(c)
    a=max(m.items(), key=operator.itemgetter(1))    
    b=max(n.items(), key=operator.itemgetter(1))
    d=list(m.values())
    h=len(text)-1
    if d[h-1]==d[h] and (d[h]==1 or d[h]==2):
        if d[h-1]==d[h-2]:
            if d[h-2]==d[h-3]:
                wt[i]=h-3
            else: wt[i]=h-2
        else: wt[i]=h-1
    else:
        wt[i]=a[0]
    wt1[i]=b[0]
    

In [312]:
wt_withbody

{55: 1,
 56: 6,
 57: 4,
 58: 3,
 59: 1,
 60: 1,
 61: 1,
 62: 3,
 63: 6,
 64: 4,
 65: 3,
 66: 6,
 67: 3,
 68: 3,
 69: 3,
 70: 6,
 71: 3,
 72: 2,
 73: 2,
 74: 6,
 75: 4,
 76: 4,
 77: 1,
 78: 1,
 79: 1,
 80: 1}

In [311]:
wt_withoutbody

{55: 4,
 56: 6,
 57: 4,
 58: 3,
 59: 4,
 60: 3,
 61: 3,
 62: 3,
 63: 6,
 64: 4,
 65: 3,
 66: 2,
 67: 3,
 68: 3,
 69: 3,
 70: 2,
 71: 3,
 72: 1,
 73: 1,
 74: 6,
 75: 6,
 76: 6,
 77: 4,
 78: 3,
 79: 3,
 80: 4}

In [29]:
def invert(d):
    return dict( (v,k) for k in d for v in d[k] )

In [30]:
c_wts=invert(C_cid_feature)

In [210]:
camp.to_csv('campfinal.csv')

In [313]:
C_wts={**c_wts,**wt_withoutbody}

In [314]:
C_wts1={**c_wts,**wt_withbody}

In [35]:
data['cwt']=0

In [138]:
data['cwt1']=0

In [39]:
test=pd.read_csv('test.csv')
test.head()

,id,campaign_id,user_id,send_date
0,63_122715,63,122715,01-02-2018 22:35
1,56_76206,56,76206,02-01-2018 08:15
2,57_96189,57,96189,05-01-2018 18:25
3,56_166917,56,166917,02-01-2018 08:15
4,56_172838,56,172838,02-01-2018 08:12


In [36]:
for key,val in c_wts.items():
    data.cwt[data['campaign_id']==key]=val

F:\Anaconda\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [40]:
test['cwt']=0

In [41]:
for key,val in wt.items():
    test.cwt[test['campaign_id']==key]=val

F:\Anaconda\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [140]:
test['cwt1']=0

In [142]:
for key,val in wt1.items():
    test.cwt1[test['campaign_id']==key]=val

F:\Anaconda\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [315]:
camp['campaign_wts']=0

In [316]:
for key,val in C_wts.items():
    camp.campaign_wts[camp['campaign_id']==key]=val

F:\Anaconda\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [317]:
camp['campaign_wts1']=0

In [318]:
for key,val in C_wts1.items():
    camp.campaign_wts1[camp['campaign_id']==key]=val

F:\Anaconda\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [949]:
camp # campaign_wts/1 are calculated after 1st iteration using email texts, first the priority order was chosen by click freq
# 6-5-4-3-2-1...high to low order and then these weights are assigned to test campaign ids by using emails subs/texts
#After 2nd iteration as shown above the campaign_weight column was added..which depicts the final weights assigned to each campaign 

,campaign_id,communication_type,total_links,no_of_internal_links,no_of_images,no_of_sections,email_body,subject,email_url,campaign_wts,campaign_wts1,images,internal_links,sections,links,external_links,campaign_weight
campaign_id,,,,,,,,,,,,,,,,,
29,29,Newsletter,67,61,12,3,"Dear AVians,\r\r\r\n \r\r\r\nWe are shaping up...",Sneak Peek: A look at the emerging data scienc...,http://r.newsletters.analyticsvidhya.com/7um44...,6,6,"(10.0, 19.0]","(26.0, 191.0]","(2.0, 6.0]","(28.0, 203.0]",6,5.40
30,30,Upcoming Events,18,14,7,1,"Dear AVians,\r\r\r\n \r\r\r\nAre your eager to...",[July] Data Science Expert Meetups & Competiti...,http://r.newsletters.analyticsvidhya.com/7up0e...,5,5,"(3.0, 10.0]","(7.0, 26.0]","(0.999, 2.0]","(9.0, 28.0]",4,4.61
31,31,Conference,15,13,5,1,Early Bird Pricing Till August 07  Save upto ...,Last chance to convince your boss before the E...,http://r.newsletters.analyticsvidhya.com/7usym...,2,2,"(3.0, 10.0]","(7.0, 26.0]","(0.999, 2.0]","(9.0, 28.0]",2,1.91
32,32,Conference,24,19,7,1,\r\r\r\n \r\r\r\nHi ?\r\r\r\n \r\r\r\nBefore I...,A.I. & Machine Learning: 5 reasons why you sho...,http://r.newsletters.analyticsvidhya.com/7uthl...,5,5,"(3.0, 10.0]","(7.0, 26.0]","(0.999, 2.0]","(9.0, 28.0]",5,4.61
33,33,Others,7,3,1,1,Fireside Chat with DJ Patil - the master is he...,"[Delhi NCR] Fireside Chat with DJ Patil, Forme...",http://r.newsletters.analyticsvidhya.com/7uvlg...,4,4,"(0.999, 3.0]","(1.999, 7.0]","(0.999, 2.0]","(5.999, 9.0]",4,3.31
34,34,Newsletter,75,64,12,4,\r\r\r\nAugust Newsletter\r\r\r\n \r\r\r\nDear...,"[AV Newsletter] Fireside Chat with DJ Patil, D...",http://r.newsletters.analyticsvidhya.com/7uwwu...,5,5,"(10.0, 19.0]","(26.0, 191.0]","(2.0, 6.0]","(28.0, 203.0]",11,4.98
35,35,Others,9,2,1,1,Fireside Chat with DJ Patil - Watch Live!\r\r\...,Watch Live: Fireside Chat with DJ Patil,http://r.newsletters.analyticsvidhya.com/7ux6c...,2,2,"(0.999, 3.0]","(1.999, 7.0]","(0.999, 2.0]","(5.999, 9.0]",7,2.20
36,36,Conference,13,11,2,1,Announcing Dr. Kirk Borne as Keynote Speaker\r...,"Dr Kirk Borne of Booz Allen Hamilton, to keyno...",http://r.newsletters.analyticsvidhya.com/7uxpa...,3,3,"(0.999, 3.0]","(7.0, 26.0]","(0.999, 2.0]","(9.0, 28.0]",2,2.41
37,37,Conference,9,7,3,1,Last day to save maximum on tickets\r\r\r\n \r...,[Last Day] Early Bird Offer on conference pass...,http://r.newsletters.analyticsvidhya.com/7uyr8...,1,1,"(0.999, 3.0]","(1.999, 7.0]","(0.999, 2.0]","(5.999, 9.0]",2,1.41


In [42]:
C_oid=order(data,'is_open',1,'campaign_id')

F:\Anaconda\lib\site-packages\ipykernel\__main__.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  from ipykernel import kernelapp as app


In [78]:
U_oid=order(data,'is_open',1,'user_id')

F:\Anaconda\lib\site-packages\ipykernel\__main__.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  from ipykernel import kernelapp as app


In [ ]:
U_oid.head()

In [79]:
U_oid=priority(U_oid,n=0)

In [82]:
2+2

4

In [81]:
test['U_oid_wts']=0

In [83]:
for key,val in U_oid.items():
    for i in val:
        test.loc[test['user_id']==i,'U_oid_wts']=key

In [ ]:
data.to_csv('output.csv')

In [ ]:
data.head()

In [46]:
U_cid=order(data,'is_click',1,'user_id')

F:\Anaconda\lib\site-packages\ipykernel\__main__.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  from ipykernel import kernelapp as app


In [47]:
U_cid=priority(U_cid,0)

In [48]:
data['U_cid_wts']=0

In [49]:
for key,val in U_cid.items():
    for i in val:
        data.loc[data['user_id']==i,'U_cid_wts']=key

In [ ]:
writer = pd.ExcelWriter('output1.xlsx') #Applying shortcut to change same column names
data.to_excel(writer,'Sheet1')
writer.save()


In [53]:
test['U_cid_wts']=0

In [92]:
data1=pd.read_csv('output.csv')

In [97]:
data.to_csv('datafinal.csv')

In [99]:
uniid=test['user_id'].unique()

In [100]:
for i in uniid:
    print(i)
    break

122715


In [109]:
len(uniid)

198219

In [102]:
d_uniid=data['user_id'].unique()

In [110]:
len(d_uniid)

168236

In [105]:
nanid=[]
for i in uniid:
    if i not in d_uniid:
        nanid.append(i)
        

In [116]:
for i in nanid:
    test.loc[test['user_id']==i,'U_oid_wts']=np.nan
    

In [119]:
for i in nanid:
    test.loc[test['user_id']==i,'U_cid_wts']=np.nan
    

In [120]:
test.to_csv('targetfinal.csv')

In [ ]:
Dat.loc[Dat['Corporate']=='Corporate','Corporate']

In [726]:
Dat=Data[Data['is_click']==1]

In [739]:
len(Dat)

12782

In [831]:
Test=pd.read_csv('test.csv')

In [833]:
len(Test)

773858

In [834]:
Train=pd.read_csv('train.csv')

In [837]:
len(Train)

1023191

In [1262]:
len(data)

1023191

In [54]:
for key,val in U_cid.items():
    for i in val:
        test.U_cid_wts[test['user_id']==i]=key

F:\Anaconda\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [84]:
test.to_csv('target.csv')

In [ ]:
P=['U_cid_wts','cwt','time_of_day','weekend',]

In [ ]:
Uid_Cid=order(data,'is_click',1,'id')

In [ ]:
len(data[data['is_open']==0])

In [ ]:
test=pd.read_csv('test.csv')

In [ ]:
camp.isnull().sum()

In [ ]:
test.user_id.max()

In [ ]:
len(data.user_id.unique())#.value_counts()

In [57]:
import datetime

In [901]:
def time(T):
    if T.weekday()<5:
        day=0
    else:
        day=1
    H=int(T.strftime('%H:%M').split(':')[0])
    if H<12 and H>6:
        time='early'
    elif (H>12 or H==12) and (H<18):
        time='afternoon'
    elif (H>18 or H==18)and H<22:
        time='evening'
    else:
        time='night'
    return day,time
    

In [902]:
dt=[];tym=[]
for i in data.send_date:
    T=datetime.datetime.strptime(i,"%d-%m-%Y %H:%M")
    day,tim=time(T)
    dt.append(day)
    tym.append(tim)

In [904]:
date=pd.DataFrame({'time_of_day':tym,'weekend':dt}, index=data.index)  

In [903]:
data.drop(['time_of_day','weekend'],axis=1,inplace=True)

In [905]:
data=pd.concat([data,date],axis=1)

In [906]:
Data=data

In [907]:
Dat=Data[Data['is_click']==1]

In [908]:
len(Dat)

12782

In [909]:
evenopen=len(Dat[Dat['time_of_day']=='evening'])

In [910]:
eventot=len(Data[Data['time_of_day']=='evening'])

In [911]:
moropen=len(Dat[Dat['time_of_day']=='early'])

In [912]:
mortot=len(Data[Data['time_of_day']=='early'])

In [913]:
nightopen=len(Dat[Dat['time_of_day']=='night'])

In [914]:
nighttotal=len(Data[Data['time_of_day']=='night'])

In [915]:
noonopen=len(Dat[Dat['time_of_day']=='afternoon'])

In [916]:
noontotal=len(Data[Data['time_of_day']=='afternoon'])

In [917]:
round(evenopen/eventot,3)

0.014

In [918]:
round(moropen/mortot,3)

0.012

In [919]:
round(nightopen/nighttotal,3)

0.011

In [920]:
round(noonopen/noontotal,3)

0.012

In [921]:
dt=[];tym=[]
for i in test.send_date:
    T=datetime.datetime.strptime(i,"%d-%m-%Y %H:%M")
    day,tim=time(T)
    dt.append(day)
    tym.append(tim)
#t=data.send_date[15544]

In [923]:
date=pd.DataFrame({'time_of_day':tym,'weekend':dt}, index=test.index)  

In [924]:
test=pd.concat([test,date],axis=1)

In [925]:
week=len(Data[Data['weekend']==0])

In [926]:
weekopen=len(Dat[Dat['weekend']==0])

In [927]:
wend=len(Data[Data['weekend']==1])

In [928]:
wendopen=len(Dat[Dat['weekend']==1])

In [929]:
ratio=(wendopen/wend)/(weekopen/week)

In [930]:
round(ratio,2)

1.42

In [931]:
round(weekopen/week,3)

0.012

In [932]:
round(wendopen/wend,3)

0.017

In [934]:
Date_day

Frequency
is_click weekend           
1        0            11402
         1             1380

In [933]:
Date_day=order(data,'is_click',1,'weekend')

F:\Anaconda\lib\site-packages\ipykernel\__main__.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  from ipykernel import kernelapp as app


In [877]:
data.drop('weekend',axis=1,inplace=True)

In [881]:
test.drop('weekend',axis=1,inplace=True)

In [935]:
Date_tym=order(data,'is_click',1,'time_of_day')

F:\Anaconda\lib\site-packages\ipykernel\__main__.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  from ipykernel import kernelapp as app


In [936]:
Date_tym

Frequency
is_click time_of_day           
1        afternoon         6184
         evening           3529
         night             1817
         early             1252

In [937]:
data.loc[data['time_of_day']=='evening','time_of_day']=0.014

In [938]:
test.loc[test['time_of_day']=='evening','time_of_day']=0.014

In [939]:
data.loc[data['time_of_day']=='night','time_of_day']=0.011

In [940]:
test.loc[test['time_of_day']=='night','time_of_day']=0.011

In [941]:
data.loc[data['time_of_day']=='early','time_of_day']=0.012

In [942]:
test.loc[test['time_of_day']=='early','time_of_day']=0.012

In [943]:
data.loc[data['time_of_day']=='afternoon','time_of_day']=0.012

In [944]:
test.loc[test['time_of_day']=='afternoon','time_of_day']=0.012

In [945]:
data.loc[data['weekend']==1,'weekend']=0.017

In [956]:
data.loc[data['weekend']==0,'weekend']=0.012

In [957]:
test.loc[test['weekend']==1,'weekend']=0.017

In [946]:
test.loc[test['weekend']==0,'weekend']=0.012

In [958]:
test.to_csv('Target.csv')

In [76]:
camp.to_csv('camp.csv')

In [4]:
data=pd.read_csv('Data_Final.csv') # final data for training

In [4]:
data.head()

,Unnamed: 0,id,user_id,campaign_id,send_date,is_open,is_click,U_cid_wts,U_oid_wts,time_of_day,...,Others,Upcoming Events,Webinar,communication_type_Conference,communication_type_Corporate,communication_type_Hackathon,communication_type_Newsletter,communication_type_Others,communication_type_Upcoming Events,communication_type_Webinar
0,0,42_14051,14051,42,01-09-2017 19:55,0,0,0,0,0.014,...,NaN,NaN,NaN,0,0,0,1,0,0,0
1,1,52_134438,134438,52,02-11-2017 12:53,0,0,0,0,0.012,...,NaN,NaN,NaN,0,0,0,1,0,0,0
2,2,33_181789,181789,33,24-07-2017 15:15,0,0,0,0,0.012,...,NaN,NaN,NaN,0,0,0,0,1,0,0
3,3,44_231448,231448,44,05-09-2017 11:36,0,0,0,0,0.012,...,NaN,NaN,NaN,0,0,0,0,0,1,0
4,4,29_185580,185580,29,01-07-2017 18:01,0,0,0,0,0.014,...,NaN,NaN,NaN,0,0,0,1,0,0,0


In [5]:
test=pd.read_csv('Test_finaL.csv') # final test data file

In [6]:
test.head()

,Unnamed: 0,id,campaign_id,user_id,send_date,U_cid_wts,U_oid_wts,time_of_day,weekend,campaign_weight,...,Others,Upcoming Events,Webinar,communication_type_Corporate,communication_type_Hackathon,communication_type_Newsletter,communication_type_Upcoming Events,communication_type_Others,communication_type_Webinar,communication_type_Conference
0,0,63_122715,63,122715,01-02-2018 22:35,0.0,0.0,0.011,0.012,5.48,...,NaN,NaN,NaN,0,0,1,0,0,0,0
1,1,56_76206,56,76206,02-01-2018 08:15,0.0,0.0,0.012,0.012,5.33,...,NaN,NaN,NaN,0,0,1,0,0,0,0
2,2,57_96189,57,96189,05-01-2018 18:25,0.0,1.0,0.014,0.012,4.36,...,NaN,NaN,NaN,0,0,0,1,0,0,0
3,3,56_166917,56,166917,02-01-2018 08:15,0.0,0.0,0.012,0.012,5.33,...,NaN,NaN,NaN,0,0,1,0,0,0,0
4,4,56_172838,56,172838,02-01-2018 08:12,0.0,2.0,0.012,0.012,5.33,...,NaN,NaN,NaN,0,0,1,0,0,0,0


In [4]:
test=pd.read_csv('Test_FinaL_nonan.csv')

In [6]:
test.columns

Index(['Unnamed: 0', 'id', 'campaign_id', 'user_id', 'send_date', 'U_cid_wts',
       'U_oid_wts', 'time_of_day', 'weekend', 'campaign_weight', 'Conference',
       'Corporate', 'Hackathon', 'Newsletter', 'Others', 'Upcoming Events',
       'Webinar', 'communication_type_Corporate',
       'communication_type_Hackathon', 'communication_type_Newsletter',
       'communication_type_Upcoming Events', 'communication_type_Others',
       'communication_type_Webinar', 'communication_type_Conference'],
      dtype='object')

In [7]:
data.columns

Index(['Unnamed: 0', 'id', 'user_id', 'campaign_id', 'send_date', 'is_open',
       'is_click', 'U_cid_wts', 'U_oid_wts', 'time_of_day', 'weekend',
       'campaign_weight', 'Conference', 'Corporate', 'Hackathon', 'Newsletter',
       'Others', 'Upcoming Events', 'Webinar', 'communication_type_Conference',
       'communication_type_Corporate', 'communication_type_Hackathon',
       'communication_type_Newsletter', 'communication_type_Others',
       'communication_type_Upcoming Events', 'communication_type_Webinar'],
      dtype='object')

In [5]:
P=['U_cid_wts','U_oid_wts','campaign_weight','communication_type_Conference',
       'communication_type_Corporate', 'communication_type_Hackathon',
       'communication_type_Newsletter', 'communication_type_Others',
       'communication_type_Upcoming Events', 'communication_type_Webinar','Conference', 'Corporate', 'Hackathon', 'Newsletter', 'Others',
       'Upcoming Events', 'Webinar','time_of_day', 'weekend']

In [6]:
Xtest=data[P]
Xtest.head()

,U_cid_wts,U_oid_wts,campaign_weight,communication_type_Conference,communication_type_Corporate,communication_type_Hackathon,communication_type_Newsletter,communication_type_Others,communication_type_Upcoming Events,communication_type_Webinar,Conference,Corporate,Hackathon,Newsletter,Others,Upcoming Events,Webinar,time_of_day,weekend
0,0,0,5.48,0,0,0,1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.014,0.012
1,0,0,5.44,0,0,0,1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.012,0.012
2,0,0,3.31,0,0,0,0,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.012,0.012
3,0,0,3.68,0,0,0,0,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.012,0.012
4,0,0,5.40,0,0,0,1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.014,0.017


In [7]:
target=data['is_click']

In [10]:
train_X, test_X, train_target, test_target= train_test_split(Xtest , target, train_size = 0.7,random_state=75)

In [14]:
test_X.shape

(306958, 19)

In [6]:
train_X=data[P]

In [7]:
train_X.shape

(1023191, 19)

In [8]:
train_target=data['is_click']

In [9]:
test_X=test[P]

In [11]:
xgtest= xgb.DMatrix(test_X.values)

In [12]:
predict=model.predict(xgtest)

In [13]:
predict=predict>0.5
predict=predict.astype(int)

In [14]:
test_target=pd.DataFrame({'is_click':predict})

In [15]:
test_target.shape

(773858, 1)

In [16]:
test_target['id']=test['id']

In [17]:
len(test_target[test_target['is_click']==1])

19334

In [18]:
(len(test_target)-len(test_target[test_target['is_click']==1]))/len(test_target[test_target['is_click']==1])

39.02575773249198

In [19]:
test_target.to_csv('submission3.csv')

In [ ]:
submission['is_click']=pd.DataFrame(test_target)

In [ ]:
submission=pd.concat([test_X.id,test_target])

In [ ]:
test_target.to_csv('submission.csv')

In [ ]:
xgb1 = XGBClassifier(learning_rate =0.1,n_estimators=3,max_depth=5,min_child_weight=1,gamma=0,subsample=0.8,colsample_bytree=0.8,
 objective= 'binary:logistic')
xgb1.fit(train_X,train_target,eval_metric='auc')

In [6]:
(len(data)-len(data[data['is_click']==1]))/len(data[data['is_click']==1])

79.04936629635425

In [ ]:
dtest_predictions = xgb1.predict(test_X)  
dtest_predprob = xgb1.predict_proba(test_X)[:,1]
#Print model report:
print("\nModel Report")
print("Accuracy on Test set: %.4g" % metrics.accuracy_score(test_target.values, dtest_predictions))
print("AUC Score on test: %f" % metrics.roc_auc_score(test_target, dtest_predprob))

In [10]:
params={'objective': 'binary:logistic','eval_metric': 'auc', 'scale_pos_weight':70,'silent':0}   
xgtrain = xgb.DMatrix(train_X.values, label=train_target.values)
#xgtest= xgb.DMatrix(test_X.values,label=test_target.values) 

watchlist = [(xgtrain, 'train')]
num_round=60

model=xgb.train(params,xgtrain,num_round,watchlist)

[0]	train-auc:0.982466
[1]	train-auc:0.985826
[2]	train-auc:0.990807
[3]	train-auc:0.99107
[4]	train-auc:0.991086
[5]	train-auc:0.991767
[6]	train-auc:0.991781
[7]	train-auc:0.993074
[8]	train-auc:0.99368
[9]	train-auc:0.993947
[10]	train-auc:0.994026
[11]	train-auc:0.994047
[12]	train-auc:0.994135
[13]	train-auc:0.994188
[14]	train-auc:0.994717
[15]	train-auc:0.995087
[16]	train-auc:0.995169
[17]	train-auc:0.9953
[18]	train-auc:0.995322
[19]	train-auc:0.995547
[20]	train-auc:0.995657
[21]	train-auc:0.995708
[22]	train-auc:0.995798
[23]	train-auc:0.995869
[24]	train-auc:0.995881
[25]	train-auc:0.995879
[26]	train-auc:0.995984
[27]	train-auc:0.996002
[28]	train-auc:0.996101
[29]	train-auc:0.996122
[30]	train-auc:0.996127
[31]	train-auc:0.996128
[32]	train-auc:0.996131
[33]	train-auc:0.99616
[34]	train-auc:0.996234
[35]	train-auc:0.996244
[36]	train-auc:0.996262
[37]	train-auc:0.996297
[38]	train-auc:0.99631
[39]	train-auc:0.996328
[40]	train-auc:0.996341
[41]	train-auc:0.996346
[42]	tra

In [11]:
params={'objective': 'binary:logistic','eval_metric': 'auc', 'scale_pos_weight':70, 'silent':0}   
xgtrain = xgb.DMatrix(train_X.values, label=train_target.values)
#xgtest= xgb.DMatrix(test_X.values,label=test_target.values) 

watchlist = [(xgtrain, 'train')]
num_round=80

model=xgb.train(params,xgtrain,num_round,watchlist)

[0]	train-auc:0.982466
[1]	train-auc:0.985826
[2]	train-auc:0.990807
[3]	train-auc:0.99107
[4]	train-auc:0.991086
[5]	train-auc:0.991767
[6]	train-auc:0.991781
[7]	train-auc:0.993074
[8]	train-auc:0.99368
[9]	train-auc:0.993947
[10]	train-auc:0.994026
[11]	train-auc:0.994047
[12]	train-auc:0.994135
[13]	train-auc:0.994188
[14]	train-auc:0.994717
[15]	train-auc:0.995087
[16]	train-auc:0.995169
[17]	train-auc:0.9953
[18]	train-auc:0.995322
[19]	train-auc:0.995547
[20]	train-auc:0.995657
[21]	train-auc:0.995708
[22]	train-auc:0.995798
[23]	train-auc:0.995869
[24]	train-auc:0.995881
[25]	train-auc:0.995879
[26]	train-auc:0.995984
[27]	train-auc:0.996002
[28]	train-auc:0.996101
[29]	train-auc:0.996122
[30]	train-auc:0.996127
[31]	train-auc:0.996128
[32]	train-auc:0.996131
[33]	train-auc:0.99616
[34]	train-auc:0.996234
[35]	train-auc:0.996244
[36]	train-auc:0.996262
[37]	train-auc:0.996297
[38]	train-auc:0.99631
[39]	train-auc:0.996328
[40]	train-auc:0.996341
[41]	train-auc:0.996346
[42]	tra

In [ ]:
params={'objective': 'binary:logistic','eval_metric': 'auc', 'scale_pos_weight':70,'gamma': 0,'silent':0}   
xgtrain = xgb.DMatrix(train_X.values, label=train_target.values)
xgtest= xgb.DMatrix(test_X.values,label=test_target.values) 

watchlist = [(xgtest, 'eval'), (xgtrain, 'train')]
num_round=51

model=xgb.train(params,xgtrain,num_round,watchlist)

[0]	eval-auc:0.982868	train-auc:0.982384
[1]	eval-auc:0.98293	train-auc:0.98269
[2]	eval-auc:0.990497	train-auc:0.990497
[3]	eval-auc:0.992276	train-auc:0.992226
[4]	eval-auc:0.992779	train-auc:0.992634
[5]	eval-auc:0.992964	train-auc:0.992762
[6]	eval-auc:0.993704	train-auc:0.993605
[7]	eval-auc:0.993736	train-auc:0.993632
[8]	eval-auc:0.993761	train-auc:0.993663
[9]	eval-auc:0.993831	train-auc:0.99373
[10]	eval-auc:0.993838	train-auc:0.993729
[11]	eval-auc:0.993846	train-auc:0.993738
[12]	eval-auc:0.994012	train-auc:0.993821
[13]	eval-auc:0.99402	train-auc:0.993827
[14]	eval-auc:0.994045	train-auc:0.99386
[15]	eval-auc:0.99418	train-auc:0.993988
[16]	eval-auc:0.995371	train-auc:0.99523
[17]	eval-auc:0.995728	train-auc:0.99562
[18]	eval-auc:0.995885	train-auc:0.995817
[19]	eval-auc:0.99597	train-auc:0.995897
[20]	eval-auc:0.996055	train-auc:0.995989
[21]	eval-auc:0.996086	train-auc:0.996046
[22]	eval-auc:0.996105	train-auc:0.996069
[23]	eval-auc:0.99615	train-auc:0.996114
[24]	eval-au

In [ ]:
params={'objective': 'binary:logistic','eval_metric': 'auc', 'scale_pos_weight':70,'gamma': 0,'silent':0}   
xgtrain = xgb.DMatrix(train_X.values, label=train_target.values)
xgtest= xgb.DMatrix(test_X.values,label=test_target.values) 

watchlist = [(xgtest, 'eval'), (xgtrain, 'train')]
num_round=70

model=xgb.train(params,xgtrain,num_round,watchlist)

[0]	eval-auc:0.982092	train-auc:0.982577
[1]	eval-auc:0.985448	train-auc:0.985936
[2]	eval-auc:0.987472	train-auc:0.987921
[3]	eval-auc:0.990695	train-auc:0.990941
[4]	eval-auc:0.993448	train-auc:0.993621
[5]	eval-auc:0.994144	train-auc:0.994305
[6]	eval-auc:0.994682	train-auc:0.994849
[7]	eval-auc:0.994495	train-auc:0.994649
[8]	eval-auc:0.994653	train-auc:0.994821
[9]	eval-auc:0.994591	train-auc:0.994773
[10]	eval-auc:0.994597	train-auc:0.994792
[11]	eval-auc:0.994551	train-auc:0.994723
[12]	eval-auc:0.994796	train-auc:0.99502
[13]	eval-auc:0.994879	train-auc:0.995111
[14]	eval-auc:0.994927	train-auc:0.99516
[15]	eval-auc:0.995207	train-auc:0.995415
[16]	eval-auc:0.995229	train-auc:0.995451
[17]	eval-auc:0.995211	train-auc:0.995446
[18]	eval-auc:0.995252	train-auc:0.99546
[19]	eval-auc:0.995609	train-auc:0.995761
[20]	eval-auc:0.995779	train-auc:0.995923
[21]	eval-auc:0.995865	train-auc:0.995997
[22]	eval-auc:0.995885	train-auc:0.996011
[23]	eval-auc:0.995916	train-auc:0.99606
[24]	e